In [1]:
import seaborn as sns
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_formats = ['pdf']

In [2]:
filename = 'grid4rand.json'
with open(filename) as json_file:
    data = json.load(json_file)
testRuns = pd.io.json.json_normalize(data)
testRuns = testRuns.loc[testRuns['tripGenRate'] < 0.05]
testRuns = testRuns.loc[testRuns['tripGenRate'] > 0.00]
testRunsComp = testRuns.loc[testRuns['lastTripStep'] > 3000]
#print(testRuns.loc[0])
testRuns

FileNotFoundError: [Errno 2] No such file or directory: 'grid4rand.json'

In [ ]:
df = testRuns
numRoadCells = (16*grids*2*(grids*2+1)*2 + (16*grids*2*grids*2)) * 2
df['averageSpeedWithPassenger'] = df['distWithPassenger'].div(df['stepsWithPassenger'])
df['averageSpeedWithoutPassenger'] = df['distWithoutPassenger'].div(df['stepsWithoutPassenger'])
df['averageSpeed'] = (df['distWithPassenger'].add(df['distWithoutPassenger'])).div(df['stepsWithoutPassenger'].add(df['stepsWithPassenger']))
df['averageStepsWithoutPassenger'] = df['stepsWithoutPassenger'].div(df['numVehicles'])
df['averageStepsTravelingWithoutPassenger'] = df['stepsTravelingWithoutPassenger'].div(df['numVehicles'])
df['densityCell'] = df['numVehicles'].div(numRoadCells)
df['densityKm'] = df['numVehicles'].div(numRoadCells)/7.5*1000
densityLabelsCell = np.around(np.arange(0.1, 0.575, 0.025), 3)
densityBinsCell = np.arange(0.075, 0.575, 0.025)
densityLabelsKm = np.arange(20, 75, 5)
densityBinsKm = np.arange(15, 75, 5)
df['densityBin'] = pd.cut(df['densityKm'], densityBinsKm, labels = densityLabelsKm)
testRuns = df
32*grids*grids/numRoadCells/7.5*1000

In [ ]:
grids = 4
numRoadCells = (16*grids*2*(grids*2+1)*2 + (16*grids*2*grids*2)) * 2
df = testRunsComp.pivot('seed','lanePolicy','averageStepsPerTrip').add_prefix('steps')
df['stepsDiff'] = df['stepsTrue'].sub(df['stepsFalse'])
df['stepsPercent'] = df['stepsDiff'].div(df['stepsFalse'])*100
df['density'] = testRunsComp.drop_duplicates('seed').set_index('seed')['density']
df['tripGenRate'] = testRunsComp.drop_duplicates('seed').set_index('seed')['tripGenRate']
df['numVehicles'] = testRunsComp.drop_duplicates('seed').set_index('seed')['numVehicles']
df['densityCell'] = df['numVehicles'].div(numRoadCells)
df['densityKm'] = df['numVehicles'].div(numRoadCells)/7.5*1000
densityLabelsCell = np.around(np.arange(0.1, 0.575, 0.025), 3)
densityBinsCell = np.arange(0.075, 0.575, 0.025)
densityLabelsKm = np.arange(20, 75, 5)
densityBinsKm = np.arange(15, 75, 5)
df['densityBin'] = pd.cut(df['densityKm'], densityBinsKm, labels = densityLabelsKm)
df = df.reset_index().rename_axis(None, axis=1)
#print (df)
travelTimeDf = df

In [ ]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "stepsPercent", data=travelTimeDf,
                facecolor='0.9', edgecolor='0.0')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Travel Time Difference (%)")
#plt.xticks(rotation=-90)
plt.show(g)

In [ ]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "averageSpeedWithPassenger", data=testRuns, hue='lanePolicy')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Average Speed with Passenger (km/h)")
#plt.xticks(rotation=-90)
plt.show(g)

In [ ]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "averageSpeed", data=testRuns, hue='lanePolicy')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Average Speed (km/h)")
#plt.xticks(rotation=-90)
plt.show(g)

In [ ]:
plt.figure(figsize=(5,3.5))
#plt.rcParams.update({'font.family': 'Times New Roman'})
g = sns.barplot("densityBin", "averageSpeedWithoutPassenger", data=testRuns, hue='lanePolicy')
plt.xlabel("Density (veh/km/lane)")
plt.ylabel("Average Speed with Passenger (km/h)")
#plt.xticks(rotation=-90)
plt.show(g)

In [ ]:
plt.figure(figsize=(5,3.5))
g = sns.scatterplot("density", "averageSpeed", data=testRuns, color='0.4', style='lanePolicy')
#g.set(xlim=(0,50))
#plt.legend(title="Lane Policy")
plt.xlabel("Density")
plt.ylabel("Travel Time (steps)")
plt.show(g)

In [ ]:
plt.figure(figsize=(5,3.5))
g = sns.scatterplot("density", "averageSpeedWithPassenger", data=testRuns, color='0.4', style='lanePolicy')
#g.set(xlim=(0,50))
#plt.legend(title="Lane Policy")
plt.xlabel("Density")
plt.ylabel("Travel Time (steps)")
plt.show(g)